### Demonstration of Stern-Gerlach simulator of a Harmonic Oscillator System with alternating measurements of position and energy

Brief outline of the simulation:

- System is initialized as a Gaussian wavepacket localized about a position $x_0$ with average momentum $\hbar k_0$

$$\Psi(x, t=0) = \frac{1}{\sigma \sqrt{2 \pi}} \: {\rm exp} \left( -\frac{1}{2} \left( \frac{x- x_0}{\sigma} \right)^2 \right) \: {\rm exp} \left(i k_0 x \right) $$

- The wavepacket is expanded in the basis of Harmonic Oscillator energy eigenfunctions permitting time-evolution of the initial state

$$ \Psi(x, t) = \sum_{n=0}^{n_{max}} c_n \psi_n(x,t) $$

where $$ \psi_n(x,t) = N_n H_n\left( \sqrt{\alpha} x \right) {\rm exp}\left(-\alpha \frac{x^2}{2} \right) {\rm exp}\left( -\frac{i}{\hbar} E_n t  \right), $$

the normalization constant is $N_n = \left(\frac{1}{2^n n!}\right)^{1/2} \left(\frac{\alpha}{\pi} \right)^{1/4} $,
$H_n(\sqrt{\alpha}x)$ is the $n^{th}$ Hermite polynomial, and $\alpha = \sqrt{\frac{\mu k}{\hbar^2}}$,
$\mu$ is the reduced mass and $k$ is the force constant.  The energy eigenvalues are given by 

$$ E_n = \hbar \sqrt{\frac{k}{\mu}} \left(n + \frac{1}{2} \right). $$

where the coefficients are determined by 

$$ c_n = \int_{-\infty}^{\infty} \psi^*_n(x,t=0) \Psi(\phi, t=0) \: dx. $$


- Measurement of position at $t = 50$ is simulated by choosing a random $x$ value weighted by the probability density; we will call the measured position $x_{collapse}$.

$$ P(x, 50) = \left|\Psi(x, 50)\right|^2 = \left| \sum_{n=0}^{n_{max}} c_n N_n H_n \left( \sqrt{\alpha} x \right) {\rm exp}\left( -\alpha \frac{x^2}{2}\right) \: {\rm exp}\left( -\frac{i}{\hbar} E_n \cdot 50  \right) \right|^2  $$

- The resulting wavefunction should be a position eigenstate located at $x = x_{collapse}$, which is approximated by a Gaussian wavepacket with a small spread in $x$ and zero average angular momentum.

- New expansion coefficients $d_m$ for the collapsed state are determined, and again the time evolution is determined from the known time-dependence of each energy eigenfunction

- Measurement of energy at $t=150$ is simulated by choosing a random quantum number $n$ weighted by the probabilities encoded by $P(n) = \left| d_n \right|^2$, resulting in an energy eigenvalue $E_n$ and collapse to an energy eigenstate 

In [ ]:
### Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
import math
from numpy.polynomial.hermite import *

### Parameters for the initial state, which will
### be a Gaussian wavepacket centered
### at x0 with spread given by sig
### and average momentum given by k0
### all values in atomic units
x0 = 1.0
sig = 0.5
k0 = 0.5

### More constants
# hbar in atomic units
hbar = 1
# pi
pi = np.pi
# force constant in atomic units
k = 1.
# reduced mass in atomic units
mu = 300.

### parameters that define the 
### resolution of the spacial grid
### and the number of energy eigenfunctios
### to use in the expansion of any given wavefunction
### throughout the simulation
x_grid_points = 200

# maximum quantum number determines the
# highest energy eigenfunction that will
# be used in the expansion
max_n = 50
# number of basis functions will be max_n + 1
# since the ground state energy eigenfunction has n = 0
# for the harmonic oscillator
num_basis_functions = max_n + 1

### Establish x array and initialize plot objects
x = np.linspace(-5,5,x_grid_points)
fig, ax = plt.subplots()
plt.close()

ax.set_xlim(( -5, 5))
ax.set_ylim((-2, 2))

line, = ax.plot([], [], lw=2)
pot,  = ax.plot([], [], lw=2)

def init():
    line.set_data([], [])
    pot.set_data([], [])
    return (line, pot,)

# function that returns an array containing the Gaussian 
# wavepacket values evaluated along x
def gaussian_packet(x0, sig, k0, x):
    ci = 0+1j
    T1 = 1/(sig * np.sqrt(2 * np.pi))
    T2 = np.exp(-0.5 * ((x-x0)/sig)**2)
    T3 = np.exp(ci * k0 * x)
    return T1 * T2 * T3

# function that returns the harmonic potential 
# evaluated along x
def harmonic_potential(k, x):
    return 0.5 * k * x ** 2

# function that returns a model of a position 
# eigenfunction, which is just a narrow gaussian function
# with avg momentum equal to zero
def position_eigenfunction(x0, sig, x):
    ci = 0+1j
    T1 = 1/(sig * np.sqrt(2 * np.pi))
    T2 = np.exp(-0.5 * ((x-x0)/sig)**2)
    return T1 * T2 

#
def energy_eigenfunction(n, k, mu, x):
    state = int(n)
    w = np.sqrt(k/mu)
    psi_ho = np.zeros_like(x)
    herm_coeff = []
    
    # the numpy hermite polynomial function hermval takes
    # an array of coefficients specifying the weight of the 
    # order of the polynomial you want... since we want 
    # only the hermite polynomial of order n corresponding to
    # the quantum number, we want this to be an array of
    # zeros except for the entry corresponding to the order n, which
    # will have the value 1
    for i in range(state):
        herm_coeff.append(0)
    herm_coeff.append(1)
    
    for i in range(0,len(x)): # in xgrid:
        psi_ho[i] = math.exp(-mu*w*x[i]**2/(2*hbar)) * hermval((mu*w/hbar)**0.5 * x[i], herm_coeff)
        
    psi_ho = np.multiply(psi_ho, 1 / (math.pow(2, state) * math.factorial(state))**0.5 * (mu*w/(pi*hbar))**0.25)
    
    return psi_ho

def energy_eigenvalue(n, k, mu):
    return np.sqrt(k/mu) * (n+(1./2))

def time_component(n, k, mu, t):
    ci = 0+1j
    En = energy_eigenvalue(n, k, mu)
    return np.exp(-ci*En*t) 


# initialize wavefunction as a gaussian wavepacket
Psi_gp = gaussian_packet(x0, sig, k0, x)

# numpy array that will be used to store
# the version of the wavefunction expanded
# in the harmonic oscillator energy eigenfunctions
y_exp = np.zeros_like(Psi_gp)

# array of quantum numbers for the expansion of 
# the wavefunction in terms of energy eigenfunctions
n_array = np.linspace(0, max_n, num_basis_functions)

# array of expansion coefficients
cn_array = np.zeros(num_basis_functions, dtype=complex)

### because the energy eigenfunctions are slightly expensive to compute, 
### we will compute them once and store them for future use...
### remember the basis functions don't change, only the coefficients!
psi = np.zeros((num_basis_functions, x_grid_points))

# expand the initial wavefunction in the basis of energy eigenfunctions
for i in range(0, num_basis_functions):
    psi[i,:] = energy_eigenfunction(n_array[i], k, mu, x)
    integrand = np.conj(psi[i,:])*Psi_gp
    cn_array[i] = np.trapz(integrand, x)
    y_exp = y_exp + cn_array[i] * psi[i,:]
    
    
# number of time-steps in the simulation
N_time = 200
# array to store the quantum number resulting from energy measurement
n0 = []


def animate(i):
    y = np.zeros(len(x),dtype=complex)
    p_of_x = harmonic_potential(k, x)
    
    if i<50:
        for j in range(0,num_basis_functions):
            ft = time_component(n_array[j], k, mu, i)
            y = y + cn_array[j] * psi[j,:] * ft

    elif i==50:
        for j in range(0,num_basis_functions):
            ft = time_component(n_array[j], k, mu, (i-50))
            y = y + cn_array[j] * psi[j,:] * ft

        P = np.real(np.conj(y) * y)

        norm = np.sum(P)
        P_norm = P / norm

        p0 = np.random.choice(x, 1, p=P_norm)
        print(" Position measured to be at ",p0)
        pf = position_eigenfunction(p0[0], 0.2, x)
        y = pf

        for j in range(0,num_basis_functions):
            integrand = np.conj(psi[j,:])*pf
            cn_array[j] = np.trapz(integrand, x)
            ft = time_component(n_array[j], k, mu, (i-50))
            y = y + cn_array[j] * psi[j,:]

    elif i<150:
        for j in range (0,num_basis_functions):
            ft = time_component(n_array[j], k, mu,  (i-50))
            y = y + cn_array[j] * psi[j,:] * ft  

    elif i==150:
        pn = np.real(np.conj(cn_array) * cn_array)
        norm = np.sum(pn)
        pn_norm = pn/norm
        nval = np.random.choice(n_array, 1, p=pn_norm)
        n0.append(int(nval[0]))
        En0 = energy_eigenvalue(n0[0], k, mu)
        print(" Randomly measured state", n0, "which has energy ",En0)
        ft = time_component(n0[0], k, mu, (i-150))
        y = psi[n0[0],:] * ft

    else:
        ft = time_component(n0[0], k, mu, i-150)
        y = psi[n0[0],:] * ft 
   
    line.set_data(x, np.real(y))
    pot.set_data(x, p_of_x)
    
    return (line, pot,)
   
anim = animation.FuncAnimation(fig, animate, init_func=init,
                        frames=N_time, interval=100, blit=True)

rc('animation', html='jshtml')
anim